In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [5]:
spark = SparkSession.builder.appName('TransformationFlagSpark').getOrCreate()

In [6]:
df_flag = spark.read.csv('storage/PRECOS SEMESTRAIS - AUTOMOTIVOS.csv', header=True, inferSchema=True, sep=';')

distinct_flag = df_flag.select('Bandeira').distinct().collect()

for row in distinct_flag:
    flag = row['Bandeira']
    df_flag_filter = df_flag.filter(col('Bandeira') == flag)
    df_flag_filter = df_flag_filter.withColumn('Data da Carga ETL', current_date())
    df_flag_filter = df_flag_filter.withColumn('Valor de Venda', regexp_replace(col('Valor de Venda'), ',', '.'))
    
    #Exibe registros que foram segregados por Bandeira 
    df_flag_filter.show()
    #Salva registros segregandos por Bandeira
    df_flag_filter.write.csv(f'storage/flag/{flag}/full/', header=True, mode="overwrite")

    #Agregando registros por Bandeira e demais
    df_flag_agg = df_flag_filter.groupBy(['Bandeira', 'Data da Coleta', 'Produto']).agg(round(avg('Valor de Venda'), 2).alias('Valor de Venda'))
    #Exibe registros que foram agregados
    df_flag_agg.show()
    #Salva registros agregados por Bandeira
    df_flag_agg.write.csv(f'storage/flag/{flag}/agg/', header=True, mode="overwrite")

spark.stop()

+--------------+--------------+----------+--------------------+-------------------+--------------------+----------+--------------------+------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+-----------------+
|Regiao - Sigla|Estado - Sigla| Municipio|             Revenda|    CNPJ da Revenda|         Nome da Rua|Numero Rua|         Complemento|      Bairro|      Cep|           Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|Data da Carga ETL|
+--------------+--------------+----------+--------------------+-------------------+--------------------+----------+--------------------+------------+---------+------------------+--------------+--------------+---------------+-----------------+--------+-----------------+
|            NE|            MA|  SAO LUIS|PIT STOP DERIVADO...| 03.001.934/0001-36|AVENIDA EDSON BRA...|        10|                null|        ANIL|65045-380|        DIESEL S10|    03/01/20